In [ ]:
import pandas as pd
import numpy as np
from scipy import interpolate
import plotly.graph_objects as go

## Loading Data

In [ ]:
filename = '../data/2020-03-10_QPD-Tilt-Calibration_Test1.csv'
col_names = ['theta_x', 'theta_y', 'qpd_x', 'qpd_y', 'qpd_sum']
df = pd.read_csv(filename, names=col_names)

### Check Data Validity

In [ ]:
df.head()

## Interpolate calibration data onto regular grid
(Not actually needed)

In [ ]:
x = df['theta_x']
y = df['theta_y']
z = df['qpd_x']

points = np.array([df['theta_x'], df['theta_y']]).transpose()
#print(points.shape)

values = df['qpd_x']
#print(values.shape)

grid_x, grid_y = np.mgrid[-0.5:0.5:100j, -0.4:0.9:100j]
#print(grid_x.shape)

znew = interpolate.griddata(points, values, (grid_x, grid_y), method='linear')
#print(znew.shape)

fig = go.Figure()

fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers'))


fig.add_trace(go.Surface(z=znew, x=grid_x, y=grid_y))
fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

## Surface Fitting
Fit 2D Functions of form $q_x=f(\theta_x, \theta_y)$ and $q_y=f(\theta_x, \theta_y)$

e.g. using 3rd order polynomials:

$q_x = a_1.\theta_x^3 + a_2.\theta_y^3 + a_3.\theta_x^2 + a_4.\theta_y^2 + a_5.\theta_x + a_6.\theta_y + a_7$

$q_y = b_1.\theta_x^3 + b_2.\theta_y^3 + b_3.\theta_x^2 + b_4.\theta_y^2 + b_5.\theta_x + b_6.\theta_y + b_7$

In [ ]:
X = x.to_numpy()
Y = y.to_numpy()
Z = z.to_numpy()

A = np.array([X**3, Y**3, X**2, Y**2, X, Y, X*0+1]).T
#print(A.shape)

coeff, r, rank, s = np.linalg.lstsq(A, B, rcond=None)

print(coeff)

In [ ]:
def poly2Dreco(X, Y, c):
    return (c[0]*X**3 + c[1]*Y**3 + c[2]*X**2 + c[3]*Y**2 + c[4]*X + c[5]*Y + c[6])

zfit = poly2Dreco(grid_x, grid_y, coeff)

fig = go.Figure()

fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers'))


fig.add_trace(go.Surface(z=zfit, x=grid_x, y=grid_y))
                  
fig.update_layout(scene = dict(
                    xaxis_title='X AXIS TITLE',
                    yaxis_title='Y AXIS TITLE',
                    zaxis_title='Z AXIS TITLE'))
fig.show()


## Calibration Function

### Populate Matrix Equation


$\begin{bmatrix}
a_{1} & a_{2} & a_{3} & a_{4} & a_{5} & a_{6} \\ 
b_{1} & b_{2} & b_{3} & b_{4} & b_{5} & b_{6}  
\end{bmatrix}
\begin{bmatrix}
\theta_x^3 \\
\theta_y^3 \\  
\theta_x^2 \\
\theta_y^2 \\  
\theta_x \\
\theta_y
\end{bmatrix}
=
\begin{bmatrix}
q_x - a_7\\
q_y - b_7
\end{bmatrix}
$

This is a matrix equation of form $Ax=b$

Populate the matrix $A$ in equation equation $Ax=b$ with the polynomial coefficients

ISSUE - WANT TO SOLVE FOR THETA_X AND THETA_Y NOT ALL 6

### Solve Matrix Equation